# Getting available actions
Actions are configured in `app\configurations` directory. When the UI makes a request to get the actions available for an object, the `app/ajaxviews/actions.py` loads the yaml, and interprets what actions that the selected object is available to take. Those actions are sent to the user. 

### Two times when actions come up:
* web interaction that creates jobs
* azure functions that resolve actions

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append("../..")
sys.path.append("..")

import yaml, ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio

# this is required for running in a Jupyter Notebook.
nest_asyncio.apply()

import sys, os

from app.connectors import cmdb_graph
from app.objects import time as t

import app.ajaxviews.actions as actions


c = cmdb_graph.CosmosdbClient()

from helpers import test_queries

executing local windows deployment


## web interaction that creates jobs

`app/ajaxviews/actions.py` takes input from the browser and creates jobs. Tools exist in that library to:
* validate which jobs an agent can do
* build the job and action graph elelements
* update the graph with the job

the resulting addition will be some kind of variant of:
`(action)<-job-(agent)`


In [2]:
# The pop here is our agent. We'll just grab one randomly. 
pop = test_queries.get_random_pop(c,'8d5b667f-b225-4641-b499-73b77558ff86')
pop


{'name': 'Istorly Gen',
 'objid': '5197028981570',
 'conformity': 0.456,
 'literacy': 0.607,
 'aggression': 0.617,
 'constitution': 0.547,
 'health': 0.7,
 'isIn': '1139503249520',
 'industry': 0.582,
 'wealth': 0.5945,
 'factionLoyalty': 0.429,
 'isIdle': 'true',
 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
 'objtype': 'pop',
 'id': '5197028981570'}

You can load the full actions list. The actions is kept in a Yaml file. This comes from `app/configurations`. It's only used in the Ajax views as the action is stored in the graph, so no need to copy to the funciton. 

In [3]:
all_actions = actions.get_actions_config()
pd.DataFrame(all_actions)

,type,label,applies_to,effort,requires_attr,augments_self_properties,description
0,healthcare_initiatives,action,pop,2,{'wealth': 0.1},"{'health': 0.05, 'wealth': '-.1'}",Spend wealth to increase the health of your po...
1,patriot_education,action,pop,1,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.1, 'agg...",expand public education programs with a collec...
2,patriot_propoganda,action,pop,1,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt..."
3,build_infrastructure,action,pop,3,{'wealth': 0.1},"{'wealth': -0.1, 'industry': 0.1}",increase industry by investing in local infras...
4,test_action,action,pop,1,{'wealth': 200},"{'wealth': -200, 'health': 0.1}",Test action for debugging purposes. Should nev...


`validator.validate()` returns the shotened list. of the actions that the population (or agent) can take. 

In [4]:
validator = actions.ActionValidator(pop, all_actions)
validator.validate()

Action Validator doesn't return anything, but populates `valid_actions` and `invalid_actions`

In [5]:
pd.DataFrame(validator.valid_actions)

,type,label,applies_to,effort,requires_attr,augments_self_properties,description
0,healthcare_initiatives,action,pop,2,{'wealth': 0.1},"{'health': 0.05, 'wealth': '-.1'}",Spend wealth to increase the health of your po...
1,patriot_education,action,pop,1,{'factionLoyalty': 0.1},"{'factionLoyalty': 0.05, 'literacy': 0.1, 'agg...",expand public education programs with a collec...
2,patriot_propoganda,action,pop,1,NaN,"{'factionLoyalty': 0.1, 'conformity': 0.1, 'ag...","build national pride, increasin faction loyalt..."
3,build_infrastructure,action,pop,3,{'wealth': 0.1},"{'wealth': -0.1, 'industry': 0.1}",increase industry by investing in local infras...


In [6]:
pd.DataFrame(validator.invalid_actions)

,type,label,applies_to,effort,requires_attr,augments_self_properties,description,rejection
0,test_action,action,pop,1,{'wealth': 200},"{'wealth': -200, 'health': 0.1}",Test action for debugging purposes. Should nev...,agent wealth is less than the required value: ...


## Creating a job

to create a job you need an agent and and action. This is done in the ajax views for that class (e.g. `app/ajaxviews/pop`) because they are a request from the UI.



In [7]:
def create_job(pop, action, universalTime):
    if type(universalTime) == list:
        universalTime = universalTime[0]
    time_to_complete = int(universalTime["currentTime"]) + int(action["effort"])
    # not actually creating the node here, just demo.
    # uid = create_action_node(c,action,pop)
    popToAction = {
        "node1": pop["objid"],
        "node2": "1234567890",
        "label": "takingAction",
        "name": "takingAction",
        "weight": time_to_complete,
        "actionType": action["type"],
        "status": "pending",
    }
    edges = [popToAction]
    return edges


time = {"currentTime": 10000}
create_job(pop, all_actions[0], time)

[{'node1': '5197028981570',
  'node2': '1234567890',
  'label': 'takingAction',
  'name': 'takingAction',
  'weight': 10002,
  'actionType': 'healthcare_initiatives',
  'status': 'pending'}]

In [8]:
time = t.Time(c)

In [9]:
time.get_global_actions()
time.actions

[]

All actions require both an agent, job and action. The properties of the action determine how it affects other things. 


### Augments self properties
Takes a dictionary of properties and augments them. Adds the property if it does not exist. 
